In [ ]:
# !pip install gspread gspread-formatting gspread_dataframe oauth2client
# !pip install gspread-formatting
# %pip install -r requirements.txt

In [ ]:
location = None
print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>', location)

In [ ]:
from google.oauth2.service_account import Credentials
import gspread
import pandas as pd
import gspread_dataframe
import numpy as np
from datetime import datetime
import calendar
import time
import json

In [ ]:
# Load the configuration from the JSON file
with open('config.json', 'r') as f:
    config = json.load(f)

# Access values from the loaded config
if  location not in ['blr', 'mum', 'pun']:
    raise ValueError('Invalid Parameter')

FOR_CURRENT_MONTH = config['FOR_CURRENT_MONTH']
config = config[location]

sheet_name = config['sheet_name']
worksheet_name = config['worksheet_name']
master_sheet_name = config['master_sheet_name']
master_worksheet_name = config['master_worksheet_name']
log_sheet_name = config['log_sheet_name']
log_worksheet_name = config['log_worksheet_name']
log_attendance_worksheet_name = config['log_attendance_worksheet_name']


**Authorize Google Sheet**

In [39]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
scopes= [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive",
    "https://spreadsheets.google.com/feeds"
]

# Authenticate google sheet with service account
is_auth = Credentials.from_service_account_file('login_cred.json', scopes=scopes)
gc = gspread.authorize(is_auth)

**Generate Log Report**

In [41]:
# Open google sheet
sheet = gc.open(sheet_name)

# Get worksheet
worksheet = sheet.worksheet(worksheet_name)
cols = ['Timestamp', 'Email Address', 'Full Name', 'Select your action']

# Load sheet data into pandas dataframe with specific columns
log_resp_df = gspread_dataframe.get_as_dataframe(worksheet=worksheet, usecols=cols)

# Drop null rows
log_resp_df.dropna(inplace=True, axis=0, how='all')

# Typecast datetime column from str to datetime
log_resp_df['Timestamp'] = pd.to_datetime(log_resp_df['Timestamp'])

if FOR_CURRENT_MONTH:
    current_month = datetime.now().month
    current_year = datetime.now().year
    # print(current_month, current_year)
    log_resp_df = log_resp_df[(log_resp_df['Timestamp'].dt.month == current_month) & (log_resp_df['Timestamp'].dt.year == current_year)]
# log_resp_df

In [42]:
# Split DateTime and creates new columns as Date and Time in df
log_resp_df[['Date', 'Time']] = log_resp_df['Timestamp'].apply(lambda x: pd.Series([x.date(), x.time()]))

# Sort the values based on Date and Email
sorted_log_resp_df = log_resp_df.sort_values(by=['Date', 'Email Address'])
# sorted_log_resp_df

In [43]:
# Set Date and Email as index, pivots the datetime in two columns based on Action column
final_log_df = sorted_log_resp_df.pivot_table(index=['Date', 'Email Address'],
                                                 columns='Select your action',
                                                 values='Timestamp',
                                                 aggfunc='first')
final_log_df.reset_index(inplace=True)

# Typecast to datetime
final_log_df['Login'] = pd.to_datetime(final_log_df['Login'])
final_log_df['Logout'] = pd.to_datetime(final_log_df['Logout'])

# to_hrs = lambda x: round(x.total_seconds() / 3600, 2) if pd.notnull(x) else pd.NaT
final_log_df['TimeSpent'] = (final_log_df['Logout'] - final_log_df['Login'])
# final_log_df

**Open Master Sheet**

In [44]:
# Open google sheet
master_sheet = gc.open(master_sheet_name)

# Get worksheet
master_worksheet = master_sheet.worksheet(master_worksheet_name)

data = master_worksheet.get_all_values()
master_df = pd.DataFrame(data[1:], columns=data[0])
# master_df

# # Fetches only active candidates from master sheet
cols = ['Admit ID', 'Name ', 'Email', 'Blr LabStatus']
master_df = master_df[cols]
master_df = master_df.rename(columns={'Name ': 'Name', 'Email': 'Email Address'})
# master_df.dropna(inplace=True, axis=0, how='all')
# master_df['Blr LabStatus'] = master_df['Blr LabStatus'].astype(str)
master_df = master_df[master_df['Blr LabStatus'] == 'Active']
# master_df

**Merge master and response**

In [45]:
# Merge log response with master sheet based on active candidates
merged_df = pd.merge(master_df, final_log_df, on='Email Address', how='inner')

# Populate Date from Datetime
merged_df['Date'] = pd.to_datetime(merged_df['Date']).apply(lambda x: x.date())

# Convert timestamp to hours
to_hrs = lambda x: round(x.total_seconds() / 3600, 2) if pd.notnull(x) else pd.NaT
merged_df['TimeSpent'] = merged_df['TimeSpent'].apply(to_hrs)

# Sort values based on log response date
merged_df.sort_values(by=['Date'], inplace=True)
# merged_df

In [46]:
log_sheet = gc.open(log_sheet_name)

# Get or create Dataframe in log csv file
cols = ['Logged At', 'Log Date', "Status"]
try:
    log_worksheet = log_sheet.worksheet(log_worksheet_name)
    date_log_df = gspread_dataframe.get_as_dataframe(log_worksheet, usecols=cols)
    date_log_df.dropna(inplace=True, axis=0, how='all')
except gspread.exceptions.WorksheetNotFound:
    log_worksheet = log_sheet.add_worksheet(title=log_worksheet_name, rows=50, cols=20)
    date_log_df = pd.DataFrame(columns=cols)
except pd.errors.EmptyDataError:
    date_log_df = pd.DataFrame()
except ValueError:
    date_log_df = pd.DataFrame(columns=cols)
# date_log_df

# Fetch unique dates from df
days = merged_df['Date'].unique()

# Fetch existing logged dates from csv file
if not date_log_df.empty:
    existing_dates = pd.to_datetime(date_log_df['Log Date'].unique()).date
else:
    existing_dates = np.array([])

# Get dates to be added to master sheet
new_dates = np.setdiff1d(days, existing_dates)

if not FOR_CURRENT_MONTH:
    today = pd.Timestamp.today()
    start_of_current_month = today.replace(day=1).date()
    start_of_previous_month = (start_of_current_month - pd.DateOffset(months=1)).replace(day=1).date()
    new_dates = [date for date in new_dates if date >= start_of_previous_month]

# new_dates

In [47]:
try:
    attendance_sheet = log_sheet.worksheet(title=log_attendance_worksheet_name)
    attendance_df = gspread_dataframe.get_as_dataframe(attendance_sheet)
    attendance_df = attendance_df.loc[:, ~attendance_df.columns.str.contains('^Unnamed')]
    attendance_df.columns = attendance_df.columns.str.replace(r'\.\d+$', '', regex=True)
    attendance_df.columns = pd.MultiIndex.from_arrays([attendance_df.columns, attendance_df.iloc[0].fillna('')])
    attendance_df.drop(0, inplace=True)

except gspread.exceptions.WorksheetNotFound:
    today = datetime.today()
    year = today.year
    month = today.month

    no_of_days = calendar.monthrange(year, month)[1]
    attendance_sheet = log_sheet.add_worksheet(title=log_attendance_worksheet_name, rows=100, cols=100)

    attendance_df = master_df.copy()
    attendance_df.columns = pd.MultiIndex.from_tuples(zip(attendance_df.columns, [''] * len(attendance_df.columns)))

    # for i in range(1, no_of_days+1):
    #     date_col = datetime(year, month, i).strftime('%Y-%m-%d')
    #     attendance_df[(date_col, 'Status')] = np.nan
    #     attendance_df[(date_col, 'Remarks')] = np.nan

    gspread_dataframe.set_with_dataframe(attendance_sheet, attendance_df)

# attendance_df.dropna(inplace=True, axis=1, how='all')
attendance_df.dropna(inplace=True, axis=0, how='all')
# attendance_df

def determine_status_remarks(row):
    login_is_valid = pd.notna(row['Login']) and row['Login'] != ""
    logout_is_valid = pd.notna(row['Logout']) and row['Logout'] != ""

    if login_is_valid and logout_is_valid:
        status = 'P'
        remarks = 'All good'
    elif login_is_valid and not logout_is_valid:
        status = 'P'
        remarks = 'Logout not done'
    elif not login_is_valid and logout_is_valid:
        status = 'P'
        remarks = 'Login not done'
    else:
        status = 'A'
        remarks = 'Absent'

    return pd.Series([status, remarks])

status_df = merged_df.apply(determine_status_remarks, axis=1)
status_df.columns = ['Status', 'Remarks']
status_df = pd.concat([merged_df, status_df], axis=1)
# merged_df
pivot_df = status_df.pivot(index='Admit ID', columns='Date', values=['Status', 'Remarks'])
pivot_df = pivot_df.swaplevel(0, 1, axis=1).sort_index(axis=1, level=0)
pivot_df = pivot_df[[col for date in pivot_df.columns.levels[0] for col in [(date, 'Status'), (date, 'Remarks')]]]

pivot_df = pivot_df[new_dates]

attendance_df = attendance_df.merge(pivot_df, left_on='Admit ID', right_index=True, how='left')

for date in new_dates:
    attendance_df[(date, 'Status')] = attendance_df[(date, 'Status')].fillna('A')
    attendance_df[(date, 'Remarks')] = attendance_df[(date, 'Remarks')].fillna('Absent')

gspread_dataframe.set_with_dataframe(attendance_sheet, attendance_df)
# attendance_df

In [48]:
# Add new dates to master sheet
dates_lst = []
for date in new_dates:
    time.sleep(1)
    log_dict = {"Logged At": pd.Timestamp.now(tz='Asia/Kolkata'), "Log Date": date, "Status": "Success"}

    formated_date = date.strftime('%d-%b-%Y') # 01-Jan-2024

    # Get or create worksheet
    try:
        ws = log_sheet.worksheet(title=str(formated_date))
    except gspread.exceptions.WorksheetNotFound:
        ws = log_sheet.add_worksheet(title=str(formated_date), rows=100, cols=10)

    # Clear and populate new data to worksheet
    try:
        ws.clear()
        df = merged_df[merged_df['Date'] == date]
        # print(df)
        gspread_dataframe.set_with_dataframe(ws, df)
    except gspread.exceptions.APIError:
        log_dict['Status'] = "Failed"

    dates_lst.append(log_dict)

# Log the latest dates added to csv file
date_log_df = pd.concat([date_log_df, pd.DataFrame(dates_lst)])
gspread_dataframe.set_with_dataframe(log_worksheet, date_log_df)